In [94]:
#######################
##  Library Imports  ##
#######################
import numpy as np
import h5py
import sys
import os
from numba import njit
import matplotlib as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif']  = 'Computer Modern Roman'
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import getopt
from itertools import zip_longest
import multiprocessing as mprocs
import time as TIME
from subprocess import Popen, PIPE
from matplotlib.pyplot import cm
from functions import tc, sim_data, import_data, import_spectra_data, import_post_processing_data, import_sys_msr
from plot_functions import plot_sector_phase_sync_snaps

In [98]:
N_loc = 16
k_frac = 0.5
num_k3_sec = 24
num_k1_sec = 1


in_dir = "../Data/Working/SIM_DATA_NAVIER_AB4CN_FULL_N[{},{}]_T[0.0,0.005,40.000]_NU[3.125e-07,1,2.0]_DRAG[0,0,-2.0]_CFL[1.00]_FORC[NONE,0,1]_u0[DECAY_TURB_ALT]_TAG[Stats-Test]/".format(int(N_loc), int(N_loc))
post_file_path = in_dir + "PostProcessing_HDF_Data_SECTORS[{},{}]_KFRAC[{:0.2f}]_TAG[Stats-Test].h5".format(int(num_k3_sec), int(num_k1_sec), k_frac)

In [99]:
# -----------------------------------------
# # --------  Read In data
# -----------------------------------------
## Read in simulation parameters
sys_vars = sim_data(in_dir)

## Read in solver data
run_data = import_data(in_dir, sys_vars)

sys_msr_data = import_sys_msr(in_dir, sys_vars)

## Read in spectra data
spec_data = import_spectra_data(in_dir, sys_vars)

## Read in post processing data
post_data = import_post_processing_data(post_file_path, sys_vars, 'file')

## Number of triad types
num_triad_types = 7

## Data Functions

In [21]:
kkx = np.fft.fftfreq(sys_vars.Nx, d=1./sys_vars.Nx)
kky = np.fft.fftfreq(sys_vars.Ny, d=1./sys_vars.Ny)
print(kkx)
print(kky)
print(kkx[:sys_vars.Nx//2 + 1])
print(kky[:,np.newaxis])
kkx[:sys_vars.Nx//2 + 1]**2 + kky[:,np.newaxis]**2

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15. -16. -15. -14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.
  -4.  -3.  -2.  -1.]
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15. -16. -15. -14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.
  -4.  -3.  -2.  -1.]
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15. -16.]
[[  0.]
 [  1.]
 [  2.]
 [  3.]
 [  4.]
 [  5.]
 [  6.]
 [  7.]
 [  8.]
 [  9.]
 [ 10.]
 [ 11.]
 [ 12.]
 [ 13.]
 [ 14.]
 [ 15.]
 [-16.]
 [-15.]
 [-14.]
 [-13.]
 [-12.]
 [-11.]
 [-10.]
 [ -9.]
 [ -8.]
 [ -7.]
 [ -6.]
 [ -5.]
 [ -4.]
 [ -3.]
 [ -2.]
 [ -1.]]


array([[  0.,   1.,   4.,   9.,  16.,  25.,  36.,  49.,  64.,  81., 100.,
        121., 144., 169., 196., 225., 256.],
       [  1.,   2.,   5.,  10.,  17.,  26.,  37.,  50.,  65.,  82., 101.,
        122., 145., 170., 197., 226., 257.],
       [  4.,   5.,   8.,  13.,  20.,  29.,  40.,  53.,  68.,  85., 104.,
        125., 148., 173., 200., 229., 260.],
       [  9.,  10.,  13.,  18.,  25.,  34.,  45.,  58.,  73.,  90., 109.,
        130., 153., 178., 205., 234., 265.],
       [ 16.,  17.,  20.,  25.,  32.,  41.,  52.,  65.,  80.,  97., 116.,
        137., 160., 185., 212., 241., 272.],
       [ 25.,  26.,  29.,  34.,  41.,  50.,  61.,  74.,  89., 106., 125.,
        146., 169., 194., 221., 250., 281.],
       [ 36.,  37.,  40.,  45.,  52.,  61.,  72.,  85., 100., 117., 136.,
        157., 180., 205., 232., 261., 292.],
       [ 49.,  50.,  53.,  58.,  65.,  74.,  85.,  98., 113., 130., 149.,
        170., 193., 218., 245., 274., 305.],
       [ 64.,  65.,  68.,  73.,  80.,  89., 100.

In [45]:
def get_w(w_hat):
    return np.fft.irfft2(w_hat)

def get_u_hat_x(w_hat, k2inv, ky):
    return 1j * k2inv * ky[:, np.newaxis] * w_hat

def get_u_hat_y(w_hat, k2inv, kx):
    return -1j * k2inv * kx * w_hat

def get_u_x(w_hat, k2inv, ky):
    return np.fft.irfft2(1j * k2inv * ky[:, np.newaxis] * w_hat)

def get_u_y(w_hat, k2inv, kx):
    return np.fft.irfft2(-1j * k2inv * kx * w_hat)

def get_nonlin_r(nonlin):
    return np.fft.irfft2(nonlin)

## Test Functions

In [46]:
def compare(a, b):
    max = 0
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            max = np.maximum(np.absolute(a[i, j] - b[i, j]), max)
    print("All Close:\t{}\nL_inf Err:\t{}\nMax Abs Err:\t{}\n".format(np.allclose(a, b), np.linalg.norm(a - b, ord = np.inf), max))

In [47]:
t = 0

## Take Solver Fourier Space Vorticity As Ground Truth

In [48]:
print(sys_msr_data.kx)
print(post_data.kx)
print(sys_msr_data.ky)
print(post_data.ky)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64 -63 -62 -61 -60 -59 -58 -57
 -56 -55 -54 -53 -52 -51 -50 -49 -48 -47 -46 -45 -44 -43 -42 -41 -40 -39
 -38 -37 -36 -35 -34 -33 -32 -31 -30 -29 -28 -27 -26 -25 -24 -23 -22 -21
 -20 -19 -18 -17 -16 -15 -14 -13 -12 -11 -10  -9  -8  -7  -6  -5  -4  -3
  -2  -1]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19

In [49]:
py_what   = run_data.w_hat[t, :, :]
py_w      = get_w(py_what)
py_uhat_x = get_u_hat_x(py_what, sys_msr_data.k2Inv, sys_msr_data.ky)
py_uhat_y = get_u_hat_y(py_what, sys_msr_data.k2Inv, sys_msr_data.kx)
py_u_x    = get_u_x(py_what, sys_msr_data.k2Inv, sys_msr_data.ky)
py_u_y    = get_u_y(py_what, sys_msr_data.k2Inv, sys_msr_data.kx)
py_nonlin = get_nonlin_r(run_data.nonlin[t, :, :])

## Compare Solver Data With Python

In [76]:
print("Real Space Vorticity:")
compare(py_w * (sys_vars.Nx * sys_vars.Ny), run_data.w[t, :, :])
print()

print("Real U_x:")
compare(py_u_x * (sys_vars.Nx * sys_vars.Ny), run_data.u[t, :, :, 0])
print()

print("Real U_y:")
compare(py_u_y * (sys_vars.Nx * sys_vars.Ny), run_data.u[t, :, :, 1])
print()

print("Fourier Uhat_x:")
compare(py_uhat_x, run_data.u_hat[t, :, :, 0])
print()

print("Fourier Uhat_y:")
compare(py_uhat_y, run_data.u_hat[t, :, :, 1])
print()

Real Space Vorticity:


IndexError: index 16 is out of bounds for axis 1 with size 16

## Compare Post Data with Python

In [51]:
print("Real Space Vorticity:")
compare(py_w * (sys_vars.Nx * sys_vars.Ny), post_data.w[t, :, :])
print()

print("Real U_x:")
compare(py_u_x * (sys_vars.Nx * sys_vars.Ny), post_data.u[t, :, :, 0])
print()

print("Real U_y:")
compare(py_u_y * (sys_vars.Nx * sys_vars.Ny), post_data.u[t, :, :, 1])
print()

print("Fourier Uhat_x:")
compare(py_uhat_x, post_data.u_hat[t, :, :, 0])
print()

print("Fourier Uhat_y:")
compare(py_uhat_y, post_data.u_hat[t, :, :, 1])
print()

Real Space Vorticity:
All Close:	True
L_inf Err:	2.8843594179761567e-13
Max Abs Err:	8.881784197001252e-15


Real U_x:
All Close:	True
L_inf Err:	2.0098506192667287e-14
Max Abs Err:	7.771561172376096e-16


Real U_y:
All Close:	True
L_inf Err:	2.042810365310288e-14
Max Abs Err:	7.216449660063518e-16


Fourier Uhat_x:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0


Fourier Uhat_y:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0




## Compare Post with Solver Data

In [100]:
t = 0
print("Fourier Vorticity:")
compare(run_data.w_hat[t, :, :], post_data.w_hat[t, :, :])
print()

print("Real Space Vorticity:")
compare(run_data.w[t, :, :], post_data.w[t, :, :])
print()

print("Real U_x:")
compare(run_data.u[t, :, :, 0], post_data.u[t, :, :, 0])
print()

print("Real U_y:")
compare(run_data.u[t, :, :, 1], post_data.u[t, :, :, 1])
print()

print("Fourier Uhat_x:")
compare(run_data.u_hat[t, :, :, 0], post_data.u_hat[t, :, :, 0])
print()

print("Fourier Uhat_y:")
compare(run_data.u_hat[t, :, :, 1], post_data.u_hat[t, :, :, 1])
print()

print("Nonlinear Terms:")
compare(run_data.nonlin[t, :, :], post_data.nonlin[t, :, :])
print()


print("ky:")
print(np.allclose(sys_msr_data.ky[:], post_data.ky[:]))
print()
print("kx:")
print(np.allclose(sys_msr_data.kx[:], post_data.kx[:]))
print()

Fourier Vorticity:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0


Real Space Vorticity:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0


Real U_x:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0


Real U_y:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0


Fourier Uhat_x:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0


Fourier Uhat_y:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0


Nonlinear Terms:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0


ky:
True

kx:
True



In [101]:
print(sys_msr_data.ky)
print(sys_msr_data.kx)


[ 0  1  2  3  4  5  6  7  8 -7 -6 -5 -4 -3 -2 -1]
[0 1 2 3 4 5 6 7 8]


In [102]:
for i in range(16):
    for j in range(9):
        print(run_data.nonlin[t, i, j])
        print(post_data.nonlin[t, i, j])

0j
0j
(0.014204734422952224+0.008322770999291787j)
(0.014204734422952224+0.008322770999291787j)
(0.019102384579437574-0.01981186859152457j)
(0.019102384579437574-0.01981186859152457j)
(-0.07624387146836953-0.09937610112556516j)
(-0.07624387146836953-0.09937610112556516j)
(0.09779203929844262-0.00915677154639649j)
(0.09779203929844262-0.00915677154639649j)
(-0.0089667641213056-0.11127514195528354j)
(-0.0089667641213056-0.11127514195528354j)
0j
0j
0j
0j
0j
0j
(0.007805201377058995-0.0019142787267726132j)
(0.007805201377058995-0.0019142787267726132j)
(-0.021489991460221295+0.012736843488651416j)
(-0.021489991460221295+0.012736843488651416j)
(-0.042798923282063264+0.028251034440083016j)
(-0.042798923282063264+0.028251034440083016j)
(0.021664135595007187+0.07680812031666553j)
(0.021664135595007187+0.07680812031666553j)
(-0.08041293874015255-0.017152553118774692j)
(-0.08041293874015255-0.017152553118774692j)
0j
0j
0j
0j
0j
0j
0j
0j
(0.003863907260080117-0.014890771913012746j)
(0.003863907260

In [79]:
t = 2
# for i in range(run_data.u_hat.shape[1]):
i = 1
for j in range(run_data.u_hat.shape[2]):
    print("uh_x[{},{}] {:1.10f} {:1.10f}I\tuh_y[{},{}] {:1.10f} {:1.10f}I".format(i, j, np.real(run_data.u_hat[t, i, j, 0]), np.imag(run_data.u_hat[t, i, j, 0]), i, j, np.real(run_data.u_hat[t, i, j, 1]), np.imag(run_data.u_hat[t, i, j, 1])))

uh_x[1,0] -0.0017108836 -0.0393766578I	uh_y[1,0] 0.0000000000 0.0000000000I
uh_x[1,1] -0.0153890578 0.0160045623I	uh_y[1,1] 0.0153890578 -0.0160045623I
uh_x[1,2] 0.0010895290 -0.0137773020I	uh_y[1,2] -0.0021790581 0.0275546040I
uh_x[1,3] -0.0106086200 0.0118952723I	uh_y[1,3] 0.0318258601 -0.0356858170I
uh_x[1,4] -0.0075275280 -0.0092337406I	uh_y[1,4] 0.0301101122 0.0369349624I
uh_x[1,5] 0.0000000000 0.0000000000I	uh_y[1,5] 0.0000000000 -0.0000000000I
uh_x[1,6] 0.0000000000 0.0000000000I	uh_y[1,6] 0.0000000000 -0.0000000000I
uh_x[1,7] 0.0000000000 0.0000000000I	uh_y[1,7] 0.0000000000 -0.0000000000I
uh_x[1,8] 0.0000000000 0.0000000000I	uh_y[1,8] 0.0000000000 -0.0000000000I


## Compare All Post with Solver Data

In [54]:
print(sys_vars.ndata)

81


In [103]:
print("Fourier Vorticity:")
for t in range(sys_vars.ndata):
    compare(run_data.w_hat[t, :, :], post_data.w_hat[t, :, :])

Fourier Vorticity:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

A

In [104]:
print("Real Space Vorticity:")
for t in range(sys_vars.ndata):
    compare(run_data.w[t, :, :], post_data.w[t, :, :])


Real Space Vorticity:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

In [105]:
print("Real U_x:")
for t in range(sys_vars.ndata):
    compare(run_data.u[t, :, :, 0], post_data.u[t, :, :, 0])


Real U_x:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:

In [106]:
print("Real U_y:")
for t in range(sys_vars.ndata):
    compare(run_data.u[t, :, :, 1], post_data.u[t, :, :, 1])


Real U_y:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:

In [107]:
print("Fourier Uhat_x:")
for t in range(sys_vars.ndata):
    compare(run_data.u_hat[t, :, :, 0], post_data.u_hat[t, :, :, 0])


Fourier Uhat_x:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All 

In [108]:
print("Fourier Uhat_y:")
for t in range(sys_vars.ndata):
    compare(run_data.u_hat[t, :, :, 1], post_data.u_hat[t, :, :, 1])


Fourier Uhat_y:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All 

In [109]:
print("Nonlinear Terms:")
for t in range(sys_vars.ndata):
    compare(run_data.nonlin[t, :, :], post_data.nonlin[t, :, :])


Nonlinear Terms:
All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All Close:	True
L_inf Err:	0.0
Max Abs Err:	0.0

All

In [68]:
print(post_data.nonlin[10, :, :])

[[ 0.00000000e+00+0.j         -3.18061972e-02-0.01573696j
  -1.18840518e-01+0.0558435j  ...  0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00-0.j         -4.41056735e-02+0.00702412j
   1.50772143e-01+0.05574991j ...  0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00-0.j         -6.34001210e-02-0.00882364j
  -8.73674538e-02-0.12852988j ...  0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 ...
 [ 8.33479755e-02+0.09383329j -3.09774120e-02-0.06743929j
   9.84114028e-02-0.02341873j ...  0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [-6.96043077e-02+0.04700692j -6.54116167e-02+0.06272986j
   3.07972673e-04+0.00958105j ...  0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [-1.43111324e-02-0.03045975j  6.69225903e-03-0.00677041j
  -7.31928221e-06+0.02925561j ...  0.00000000e+00+0.j
   0.00000000e+00+0.j       

In [110]:
print(run_data.nonlin[10, :, :])

[[ 0.        +0.j          0.01318811-0.00889339j -0.03588121-0.04892222j
   0.04147013+0.00046817j -0.04601507-0.02983796j  0.01152775+0.19157481j
   0.        +0.j          0.        +0.j          0.        +0.j        ]
 [-0.00288436+0.00038952j -0.03639127-0.0203462j  -0.03777556+0.03945027j
  -0.09013621-0.06318064j -0.08657418+0.19799839j  0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j        ]
 [-0.03658277+0.04103522j  0.00496787+0.08000411j  0.08472372+0.07933783j
  -0.00070354-0.06917379j  0.0309445 -0.06536624j  0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j        ]
 [ 0.10969603-0.01078541j -0.05434882+0.04488692j  0.14096382-0.05294242j
  -0.07680659-0.09376273j -0.0504665 +0.10336725j  0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j        ]
 [ 0.02141562-0.05935099j  0.1209641 +0.13700721j  0.09915584-0.00095427j
  -0.10623565+0.1066044j   0.        +0.j          0.     